# Living Earth - Schema translation

---
<div>The aim of this notebooks is to transform and initial Level4 LCCS map to different scheme</div>

<br><i>Author(s):</i> <a href='https://www.unige.ch/envirospace/people/giuliani' target='_blank'>Gregory Giuliani</a>
<br><i>Version:</i> 0.1
<br><i>Date:</i> 2025-10-27
<br><i>Supported by:</i> SNSF <a href='https://data.snf.ch/grants/grant/224912' target='_blank'>DT4LC</a>; Horizon-Europe <a href='https://landshift.eu' target='_blank'>LandShift</a> and <a href='https://monalisa4land.eu' target='_blank'>MONALISA</a> projects

---
### Load librairies

In [16]:
import os
import rasterio
import numpy as np
from rasterio.crs import CRS

### File

In [17]:
lccsFile = '/Users/gregorygiuliani/Library/CloudStorage/OneDrive-unige.ch/projects/current/LivingEarth/schema_transform/level4_out_ch_2018.tif' #TBC
outputFld = '/Users/gregorygiuliani/Desktop/'

In [18]:
# Open the raster file
with rasterio.open(lccsFile) as src:
    band_names = src.descriptions

print('Bands: '+str(band_names))
print('CRS: '+str(src.crs))

Bands: ('level3', 'lifeform_veg_cat_l4a', 'canopyco_veg_cat_l4d_ch', 'canopyht_veg_cat_l4d', 'leaftype_veg_cat_l4a_ch', 'phenolog_veg_cat_l4a_ch', 'waterstt_wat_cat_l4a', 'snowcper_wat_cat_l4d_ch', 'baresurf_phy_cat_l4a', 'artisurf_urb_cat_l4a', 'martdens_urb_cat_l4d_ch', 'level4')
CRS: EPSG:2056


## Land Cover - NOLC04
https://www.bfs.admin.ch/bfs/fr/home/statistiques/espace-environnement/nomenclatures/arealstatistik/nolc2004.html

In [19]:
#Level 2
#output
output_raster = outputFld+'nolcL2.tif'

In [20]:
#Level 1
#output
output_raster = outputFld+'nolcL1.tif'

## Land Cover - IPCC

In [21]:
#Level1
#output
output_raster = outputFld+'ipccL1.tif'

# Open the multiband raster
with rasterio.open(lccsFile) as src:
    # Read all bands as a 3D numpy array: (bands, rows, cols)
    data = src.read()
    profile = src.profile.copy()

# Extract individual bands for readability
level3 = data[0]
lifeform = data[1]

# Initialize classification array and assign the default value 60 (other land)
classification = np.full(level3.shape, 60, dtype=np.uint8)

# Apply your conditions
classification[(level3 == 112) & (lifeform == 1)] = 10 #Forest land
classification[(level3 == 112) & (lifeform == 2)] = 20 #Grassland
classification[(level3 == 111)] = 30 #Cropland
classification[(level3 == 124)] = 40 #Wetlands
classification[(level3 == 215)] = 50 #Settlements

# Update the profile for a single-band output
profile.update(count=1, dtype=rasterio.uint8, crs=src.crs)

# Write the new classified raster
with rasterio.open(output_raster, 'w', **profile) as dst:
    dst.write(classification, 1)

print("✅ Classification complete and saved to", output_raster)

✅ Classification complete and saved to /Users/gregorygiuliani/Desktop/ipccL1.tif


## Land Cover - ESA CCI
https://climate.esa.int/en/projects/land-cover/

In [22]:
#Level 2
#output
output_raster = outputFld+'cciL2.tif'

In [23]:
#Level 1
#output
output_raster = outputFld+'cciL1.tif'

## Land Cover - CORINE
https://land.copernicus.eu/en/products/corine-land-cover

In [24]:
#Level 3
#output
output_raster = outputFld+'corineL3.tif'

In [25]:
#Level 2
#output
output_raster = outputFld+'corineL1.tif'

In [26]:
#Level 1
#output
output_raster = outputFld+'corineL1.tif'

## Land Cover - EAGLE
https://land.copernicus.eu/en/eagle

In [27]:
#Level 3
#output
output_raster = 'eagleL3.tif'

In [28]:
#Level 2
#output
output_raster = 'eagleL2.tif'

In [29]:
#Level 1
#output
output_raster = 'eagleL1.tif'

## Habitat - EUNIS
https://eunis.eea.europa.eu

In [30]:
#Level 1
#output
output_raster = outputFld+'eunisL1.tif'

## Habitat - TypoCH
https://www.infoflora.ch/en/habitats/typoch-(delarze-et-al.).html

In [31]:
#Level 2
#output
output_raster = outputFld+'typochL2.tif'

In [32]:
#Level 1
#output
output_raster = outputFld+'typochL1.tif'

## Gloabl Ecosystem Typology
https://global-ecosystems.org/explore

In [ ]:
#3. Functional groups

In [ ]:
#2. Biomes

In [ ]:
#1. Realms

## Ecosystems - MAES
https://www.eea.europa.eu/en/analysis/indicators/ecosystem-coverage-in-europe

In [33]:
#Level 2
#output
output_raster = outputFld+'maesL2.tif'

# Open the multiband raster
with rasterio.open(lccsFile) as src:
    # Read all bands as a 3D numpy array: (bands, rows, cols)
    data = src.read()
    profile = src.profile.copy()

# Extract individual bands for readability
level3 = data[0]
lifeform = data[1]
canopyco = data[2]
canopyht = data[3]

# Initialize classification array and assign the default value 0
classification = np.full(level3.shape, 0, dtype=np.uint8)

# Apply your conditions
classification[(level3 == 215)] = 101 #Urban (Settlements and other artificial areas)
classification[(level3 == 111)] = 102 #Cropland
classification[(level3 == 112) & (lifeform == 2) & (canopyco >= 13) & (canopyht >= 9)] = 103 #Grassland
classification[(level3 == 112) & (lifeform == 1) & (canopyco <= 12) & (canopyht <= 8)] = 104 #Forest and woodlands
classification[(level3 == 112) & (lifeform == 2) & (canopyco >= 13) & (canopyht >= 8)] = 105 #Heathland and shrub
classification[(level3 == 216)] = 106 #Sparsely vegetated land
classification[(level3 == 124)] = 107 #Wetlands
classification[(level3 == 227)] = 200 #Rivers and lakes
classification[(level3 == 228)] = 200 #Rivers and lakes
#301 - Marine inlets and transitional waters
#302 - Coastal
#303 - Shelf
#304 - Open Ocean

# Update the profile for a single-band output
profile.update(count=1, dtype=rasterio.uint8, crs=src.crs)

# Write the new classified raster
with rasterio.open(output_raster, 'w', **profile) as dst:
    dst.write(classification, 1)

print("✅ Classification complete and saved to", output_raster)


✅ Classification complete and saved to /Users/gregorygiuliani/Desktop/maesL2.tif


In [34]:
#Level 1
#input
input_raster = outputFld+'maesL2.tif'
#output
output_raster = outputFld+'maesL1.tif'

#Reclassify
with rasterio.open(input_raster) as src:
    # Read as numpy array
    array = src.read()
    profile = src.profile

    # Reclassify
    array[np.where(array == 101)] = 10 #Terrestrial
    array[np.where(array == 102)] = 10
    array[np.where(array == 103)] = 10
    array[np.where(array == 104)] = 10
    array[np.where(array == 105)] = 10
    array[np.where(array == 106)] = 10
    array[np.where(array == 107)] = 10
    array[np.where(array == 200)] = 20 #Freshwater
    array[np.where(array == 301)] = 30 #Marine
    array[np.where(array == 302)] = 30
    array[np.where(array == 303)] = 30
    array[np.where(array == 304)] = 30

with rasterio.open(output_raster, 'w', **profile) as dst:
    # Write to disk
    dst.write(array)

print(f"Reclassified file '{output_raster}' created successfully.")

Reclassified file '/Users/gregorygiuliani/Desktop/maesL1.tif' created successfully.
